In [1]:
# environment set up
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf # only use tensorflow keras
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd

# working folder
# directory = "/Users/Srikar/Desktop/Velexi/spectra-ml/data"
directory = os.environ['DATA_DIR']
os.chdir(directory)

# print(os.getcwd())

In [4]:
stddata_path = os.path.join(directory,"Srikar-Standardized")
metadata = pd.read_csv(os.path.join(stddata_path,"spectra-metadata.csv"), sep="|")
metadata.head()

,spectrum_id,value_type,material,spectrometer_purity_code,measurement_type,raw_data_path
0,23077,reflectance,Rangeland C03-022 S12% G22%,ASDFRa,AREF,ChapterV_Vegetation/splib07a_Rangeland_C03-022...
1,22344,reflectance,Marsh SPAL92%...a CRMS322v78,ASDFRa,AREF,ChapterV_Vegetation/splib07a_Marsh_SPAL92%...a...
2,22136,reflectance,Tumbleweed ANP92-2C Dry,BECKa,AREF,ChapterV_Vegetation/splib07a_Tumbleweed_ANP92-...
3,24017,reflectance,Rangeland L02-069 S00% G99%,ASDFRa,AREF,ChapterV_Vegetation/splib07a_Rangeland_L02-069...
4,21060,reflectance,Douglas-Fir YNP-DF-1 forest,AVIRISb,RTGC,ChapterV_Vegetation/splib07a_Douglas-Fir_YNP-D...


In [5]:
metadata.iloc[327, 1]

'errorbar'

In [6]:
# metadata.columns
metadata = metadata[~metadata['spectrometer_purity_code'].str.contains("NIC4")]
# ~metadata['spectrometer_purity_code'].str.contains("NIC4")
metadata.shape

(2488, 6)

In [7]:
x = 0
for i in range(metadata.shape[0]):
    if metadata.iloc[i, 1] == 'reflectance':
        spectrum = pd.read_csv(os.path.join(stddata_path,"{}.csv".format(metadata.iloc[i, 0])))
        for j in range(spectrum.shape[0]):
            if np.isnan(spectrum.iloc[j, 1]):
                print(str(metadata.iloc[i, 3]) + " " + str(metadata.iloc[i, 0]) + " " + str(metadata.iloc[i, 2]))
                x += 1
                break

print(x)

AVIRISb 21060 Douglas-Fir YNP-DF-1 forest
AVIRISb 21098 Espruce-Sfir YNP-SF-5 forst
AVIRISb 21594 Lodgepole-Pine YNP-LP1-A
AVIRISb 21629 Lodgepole-Pine YNP-LP3-B
ASDFRa 20883 Aspen Aspen-3 yellowGreenTop
AVIRISb 22162 WhitebarkPine YNP-WB-2 frst
AVIRISb 22072 Sagebrush YNP-SS-1
AVIRISb 21085 Espruce-Sfir YNP-SF-2 forst
AVIRISb 21081 Espruce-Sfir YNP-SF-1 forst
ASDFRa 20892 Aspen Aspen-4 yellow-top
AVIRISb 21614 Lodgepole-Pine YNP-LP2-A
AVIRISb 21579 Lodgepole-Pine YNP-LP
AVIRISb 22098 Sedge YNP-SE-1
AVIRISb 21228 Grass-FescueWheatg YNP-FW-1
AVIRISb 22150 Wetland YNP-WT-1
AVIRISb 21046 Conifer-Meadow-Mix YNP-CM-1
AVIRISb 22175 Willow YNP-WI-1
AVIRISb 20915 Aspen YNP-AS-1 forest
AVIRISb 22179 Willow-Sedge-Mix YNP-WS-1
AVIRISb 21089 Espruce-Sfir YNP-SF-3 forst
AVIRISb 21609 Lodgepole-Pine YNP-LP1-D
AVIRISb 21064 Douglas-Fir YNP-DF-2 forest
AVIRISb 21093 Espruce-Sfir YNP-SF-4 forst
AVIRISb 22087 Sagebrush YNP-SS-4
ASDFRa 21837 P.australis CRMS-0153 dryNPV
AVIRISb 22154 Wetland YNP-WT-2
ASD

In [8]:
num_nic = 0
for i in range(metadata.shape[0]):
    if metadata.iloc[i, 3].find("NIC") != -1:
        num_nic += 1

print(num_nic)

0
